In [1]:

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Load and preprocess dataset
df = pd.read_csv('calories.csv')
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

features = df.drop(['User_ID', 'Calories'], axis=1)
target = df['Calories'].values
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.1, random_state=22)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Initialize models including XGBoost
models = {
    'Linear Regression': LinearRegression(),
    'KNN Regressor': KNeighborsRegressor(),
    'SVR': SVR(),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost Regressor': XGBRegressor(objective ='reg:squarederror', random_state=42)
}

# Train & evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    val_preds = model.predict(X_val)
    mae_val = mean_absolute_error(y_val, val_preds)
    r2_val = r2_score(y_val, val_preds)
    results[name] = {'model': model, 'MAE': mae_val, 'R2': r2_val}

# Select best model based on highest R2
best_model_name = max(results, key=lambda k: results[k]['R2'])
best_model = results[best_model_name]['model']

print(f"Best model: {best_model_name} with Validation R2: {results[best_model_name]['R2']:.4f}")

Best model: XGBoost Regressor with Validation R2: 0.9990


In [3]:
!pip install gradio

In [5]:
import gradio as gr
import numpy as np

def predict_calories(gender, age, height, weight, duration, heart_rate, body_temp):
    # Preprocess input, predict with your best model, etc.
    gender_encoded = 1 if gender == 'Male' else 0
    user_features = np.array([[gender_encoded, age, height, weight, duration, heart_rate, body_temp]])
    user_features_scaled = scaler.transform(user_features)
    prediction = best_model.predict(user_features_scaled)[0]
    return round(prediction, 2), best_model_name

iface = gr.Interface(
    fn=predict_calories,
    inputs=[
        gr.Radio(['Male', 'Female'], label="Gender"),
        gr.Slider(10, 100, label="Age"),
        gr.Slider(100, 250, label="Height (cm)"),
        gr.Slider(30, 200, label="Weight (kg)"),
        gr.Slider(1, 300, label="Duration (minutes)"),
        gr.Slider(40, 200, label="Heart Rate"),
        gr.Slider(10, 110, label="Body Temperature (F)")
    ],
    outputs=[
        gr.Textbox(label="Predicted Calories Burnt"),
        gr.Textbox(label="Model Used")
    ],
    title="Calories Burnt Prediction"
)

iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57be432da8560f42ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
